# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [2]:
print(pd.__version__)

0.25.3


In [3]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=postgres password=postgres")
cur = conn.cursor()

In [4]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [27]:
song_files = get_files("data/song_data/")
song_files

['d:\\Projects 2019\\06-data-eng\\02-coursework\\de-nanodegree\\de-nano-projects\\de-nano-project-1\\data\\song_data\\A\\A\\A\\TRAAAAW128F429D538.json',
 'd:\\Projects 2019\\06-data-eng\\02-coursework\\de-nanodegree\\de-nano-projects\\de-nano-project-1\\data\\song_data\\A\\A\\A\\TRAAABD128F429CF47.json',
 'd:\\Projects 2019\\06-data-eng\\02-coursework\\de-nanodegree\\de-nano-projects\\de-nano-project-1\\data\\song_data\\A\\A\\A\\TRAAADZ128F9348C2E.json',
 'd:\\Projects 2019\\06-data-eng\\02-coursework\\de-nanodegree\\de-nano-projects\\de-nano-project-1\\data\\song_data\\A\\A\\A\\TRAAAEF128F4273421.json',
 'd:\\Projects 2019\\06-data-eng\\02-coursework\\de-nanodegree\\de-nano-projects\\de-nano-project-1\\data\\song_data\\A\\A\\A\\TRAAAFD128F92F423A.json',
 'd:\\Projects 2019\\06-data-eng\\02-coursework\\de-nanodegree\\de-nano-projects\\de-nano-project-1\\data\\song_data\\A\\A\\A\\TRAAAMO128F1481E7F.json',
 'd:\\Projects 2019\\06-data-eng\\02-coursework\\de-nanodegree\\de-nano-projects\\

In [28]:
filepath = "data/song_data/"

In [29]:
song_files[0]

'd:\\Projects 2019\\06-data-eng\\02-coursework\\de-nanodegree\\de-nano-projects\\de-nano-project-1\\data\\song_data\\A\\A\\A\\TRAAAAW128F429D538.json'

In [30]:
df=pd.read_json(song_files[0], typ='series')
df.head()

num_songs                            1
artist_id           ARD7TVE1187B99BFB1
artist_latitude                   None
artist_longitude                  None
artist_location        California - LA
dtype: object

In [31]:
df=pd.DataFrame([pd.read_json(song_files[0], typ='series')])
df

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARD7TVE1187B99BFB1,None,None,California - LA,Casual,SOMZWCG12A8C13C480,I Didn't Mean To,218.93179,0


In [32]:
df.values

array([[1, 'ARD7TVE1187B99BFB1', None, None, 'California - LA', 'Casual',
        'SOMZWCG12A8C13C480', "I Didn't Mean To", 218.93179, 0]],
      dtype=object)

In [0]:
df.dtypes

## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [33]:
song_data = df[['song_id','title','artist_id','year','duration']].values[0].tolist()
song_data

['SOMZWCG12A8C13C480', "I Didn't Mean To", 'ARD7TVE1187B99BFB1', 0, 218.93179]

In [34]:
type(song_data[4])

float

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [35]:
cur.execute(song_table_insert, song_data)
conn.commit()

In [0]:
df.dtypes

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [0]:
artist_data = df[['artist_id','artist_name','artist_location','artist_latitude','artist_longitude']].values[0].tolist()
artist_data

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [36]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [37]:
log_files = get_files('data/log_data/')
#log_files

In [38]:
filepath = 'data/log_data'

In [39]:
df=pd.read_json(log_files[0], typ='series', lines=True)
df=pd.DataFrame([df.iloc[0]])
df

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged In,Walter,M,0,Frye,None,free,"San Francisco-Oakland-Hayward, CA",GET,Home,1.540919e+12,38,None,200,1541105830796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",39


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [40]:
df=pd.read_json(log_files[0], lines=True)
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged In,Walter,M,0,Frye,NaN,free,"San Francisco-Oakland-Hayward, CA",GET,Home,1540919166796,38,None,200,1541105830796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",39
1,None,Logged In,Kaylee,F,0,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Home,1540344794796,139,None,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,You Gotta Be,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
3,None,Logged In,Kaylee,F,2,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Upgrade,1540344794796,139,None,200,1541106132796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
4,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Flat 55,200,1541106352796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8


In [41]:
t = df[df['page']=='NextSong'].copy().reset_index(drop=True)
t['epoch']=t['ts']
t['ts']=pd.to_datetime(df['ts'])

t.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId,epoch
0,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,You Gotta Be,200,1970-01-01 00:25:41.105830796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8,1541106106796
1,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Flat 55,200,1970-01-01 00:25:41.106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8,1541106352796
2,Tamba Trio,Logged In,Kaylee,F,4,Summers,177.18812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Quem Quiser Encontrar O Amor,200,1970-01-01 00:25:41.106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8,1541106496796
3,The Mars Volta,Logged In,Kaylee,F,5,Summers,380.42077,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Eriatarka,200,1970-01-01 00:25:41.106132796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8,1541106673796
4,Infected Mushroom,Logged In,Kaylee,F,6,Summers,440.26730,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Becoming Insane,200,1970-01-01 00:25:41.106352796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8,1541107053796


In [42]:
time_data=[t['ts'].values,
           t['ts'].dt.hour.values, 
           t['ts'].dt.day.values, 
           t['ts'].dt.week.values, 
           t['ts'].dt.month.values, 
           t['ts'].dt.year.values, 
           t['ts'].dt.weekday.values,
           t['epoch'].values]
            
column_labels = ['timestamp',
                 'hour',
                 'day_of_year',
                 'week_of_year',
                 'month',
                 'year',
                 'weekday',
                 'epoch']

In [43]:
time_data

[array(['1970-01-01T00:25:41.105830796', '1970-01-01T00:25:41.106106796',
        '1970-01-01T00:25:41.106106796', '1970-01-01T00:25:41.106132796',
        '1970-01-01T00:25:41.106352796', '1970-01-01T00:25:41.106496796',
        '1970-01-01T00:25:41.106673796', '1970-01-01T00:25:41.107053796',
        '1970-01-01T00:25:41.107493796', '1970-01-01T00:25:41.107734796',
        '1970-01-01T00:25:41.108520796'], dtype='datetime64[ns]'),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64),
 array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int64),
 array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int64),
 array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int64),
 array([1970, 1970, 1970, 1970, 1970, 1970, 1970, 1970, 1970, 1970, 1970],
       dtype=int64),
 array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3], dtype=int64),
 array([1541106106796, 1541106352796, 1541106496796, 1541106673796,
        1541107053796, 1541107493796, 1541107734796, 1541108520796,
        1541109125796, 1541109325796, 1541110994796

In [44]:
#time_df=pd.DataFrame(data=time_data, index=column_labels).transpose()
#time_df.head()

dic_time = dict(zip(column_labels, time_data)) # creating a dictionary
time_df = pd.DataFrame.from_dict(dic_time)
time_df.head()

,timestamp,hour,day_of_year,week_of_year,month,year,weekday,epoch
0,1970-01-01 00:25:41.105830796,0,1,1,1,1970,3,1541106106796
1,1970-01-01 00:25:41.106106796,0,1,1,1,1970,3,1541106352796
2,1970-01-01 00:25:41.106106796,0,1,1,1,1970,3,1541106496796
3,1970-01-01 00:25:41.106132796,0,1,1,1,1970,3,1541106673796
4,1970-01-01 00:25:41.106352796,0,1,1,1,1970,3,1541107053796


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [46]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [47]:
user_df = df[['userId','firstName','lastName','gender','level']].copy()

In [48]:
df.dtypes

artist            object
auth              object
firstName         object
gender            object
itemInSession      int64
lastName          object
length           float64
level             object
location          object
method            object
page              object
registration       int64
sessionId          int64
song              object
status             int64
ts                 int64
userAgent         object
userId             int64
dtype: object

In [49]:
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged In,Walter,M,0,Frye,NaN,free,"San Francisco-Oakland-Hayward, CA",GET,Home,1540919166796,38,None,200,1541105830796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",39
1,None,Logged In,Kaylee,F,0,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Home,1540344794796,139,None,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,You Gotta Be,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
3,None,Logged In,Kaylee,F,2,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Upgrade,1540344794796,139,None,200,1541106132796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
4,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Flat 55,200,1541106352796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [50]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [51]:
df['ts'] = pd.to_datetime(df['ts'])

In [52]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    results = cur.execute(song_select, (row.song, row.artist, row.length))
    songid, artistid = results if results else None, None
    # insert songplay record
    songplay_data = (index, row['ts'], row['userId'], row['level'], songid, 
                     artistid, row['sessionId'], row['location'], row['userAgent'])
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()


In [53]:
#conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=postgres password=postgres")
#cur = conn.cursor()
#cur.execute(songplay_table_insert, songplay_data)
#conn.commit()

In [54]:
df.dtypes

artist                   object
auth                     object
firstName                object
gender                   object
itemInSession             int64
lastName                 object
length                  float64
level                    object
location                 object
method                   object
page                     object
registration              int64
sessionId                 int64
song                     object
status                    int64
ts               datetime64[ns]
userAgent                object
userId                    int64
dtype: object

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [55]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.